# Pandas and SQL

# Overview

In this exercise, we split the raw table into tables corresponding the **implied subjects** we discovered. 

Then we use joins to reassemble the data. 

We will assumed that the implied subjects are ARTIST, GENRE, ALBUM, and REVIEW.

To do this, we use Pandas to store the initial table as a dataframe, and then we create variations of that dataframe and save those to the database as new tables. 

# Connect to the database

In [1]:
import sqlite3
import pandas as pd
import numpy as np

We create an empty database to store our work.

In [2]:
db = sqlite3.connect('./data/pitchfork.db')

# Get the raw review table

Get the review data.

In [3]:
raw = pd.read_csv("./data/pitchfork.csv").set_index('id')

In [4]:
raw

,album,artist,best,date,genre,review,score
id,,,,,,,
1,A.M./Being There,Wilco,1,December 6 2017,Rock,Best new reissue 1 / 2 Albums Newly reissued a...,7.0
2,No Shame,Hopsin,0,December 6 2017,Rap,"On his corrosive fifth album, the rapper takes...",3.5
3,Material Control,Glassjaw,0,December 6 2017,Rock,"On their first album in 15 years, the Long Isl...",6.6
4,Weighing of the Heart,Nabihah Iqbal,0,December 6 2017,Pop/R&B,"On her debut LP, British producer Nabihah Iqba...",7.7
5,The Visitor,Neil Young / Promise of the Real,0,December 5 2017,Rock,"While still pointedly political, Neil Youngís ...",6.7
...,...,...,...,...,...,...,...
19551,1999,Cassius,0,January 26 1999,Electronic,"Well, it's been two weeks now, and I guess it'...",4.8
19552,Let Us Replay!,Coldcut,0,January 26 1999,Electronic,The marketing guys of yer average modern megac...,8.9
19553,"Singles Breaking Up, Vol. 1",Don Caballero,0,January 12 1999,Experimental,"Well, kids, I just went back and re-read my re...",7.2


# Remove rows with missing data

Here's a quick to inventory which columns have missing data.

In [5]:
raw.isna().sum()

album     5
artist    0
best      0
date      0
genre     0
review    1
score     0
dtype: int64

In [6]:
raw[raw.album.isna()]

,album,artist,best,date,genre,review,score
id,,,,,,,
58,NaN,Gord Downie,0,November 17 2017,Rock,Recorded in his final months and released 10 d...,7.4
261,NaN,Yves Tumor,0,September 23 2017,Experimental,This self-released compilation from Yves Tumor...,7.8
273,NaN,Lee Ranaldo,0,September 20 2017,Experimental,Lee Ranaldoís latest album is his finest post-...,6.5
351,NaN,Queens of the Stone Age,0,August 28 2017,Rock,"With production flourishes from Mark Ronson, J...",6.9
6795,NaN,Inverloch,0,April 9 2012,None,The Melbourne metal band forges a strong bridg...,6.7


In [7]:
raw[raw.review.isna()]

,album,artist,best,date,genre,review,score
id,,,,,,,
13301,Shine On,Jet,0,October 2 2006,Rock,NaN,0.0


In [8]:
raw = raw.dropna()

Drop dupicate rows, too.

In [9]:
raw = raw.drop_duplicates()

In [10]:
raw = raw.reset_index().set_index('id')

In [11]:
raw.to_sql('raw', db, if_exists='replace')

19544

# Create implied subject tables

## The `artist` table

We get the unique artist names and save them to a new dataframe, and change the column name and the index name in the process so that we have well-named columns in our table. 

In [12]:
artists = raw.artist.value_counts().to_frame('n_reviews')
artists.index.name = 'artist_id'
artists.to_sql('artist', db, index=True, if_exists='replace', index_label='artist_id')

9249

In [13]:
artists.sample(10)

,n_reviews
artist_id,
Flying Canyon,1
Florence and the Machine,4
Larkin Grimm,3
Dylan LeBlanc,2
Devo,2
Tory Lanez,3
Sacred Paws,1
Little Wings,4
Boris / Ian Astbury,1


How many albums does Wilco have?

In [15]:
artists.loc['Wilco'].n_reviews

14

### Pattern

1. Import the columns that you want from raw table into a new dataframe. `value_counts()` is your friend.
2. Rename columns and index as desired.
3. Put the new dataframe into the database as a new table. No need to define the schema!

## The `genre` table

In [16]:
genres = raw.genre.value_counts().to_frame('n_reviews')
genres.index.name = 'genre_id'
genres.to_sql('genre', db, index=True, if_exists='replace', index_label='genre_id')

10

In [17]:
genres

,n_reviews
genre_id,
Rock,6954
Electronic,4020
None,2320
Experimental,1696
Rap,1481
Pop/R&B,1157
Metal,781
Folk/Country,700
Jazz,257


In [ ]:
genres.plot.barh();

## The `album` table

In [ ]:
albums = raw[['album','artist','genre']].value_counts().to_frame('n_reviews').reset_index().set_index(['artist','album'])
albums.index.names = ['artist_id','album_id']
albums = albums.rename(columns={'genre':'genre_id'})
albums.to_sql('album', db, index=True, if_exists='replace', index_label=albums.index.names)

In [ ]:
albums

## The `review` table

In [ ]:
reviews = raw[['album', 'artist', 'best', 'date', 'review', 'score']].copy()
reviews.columns = ['album_id', 'artist_id', 'is_best', 'review_date', 'review_content', 'album_score']
reviews = reviews.set_index(['artist_id','album_id','review_date'])
reviews.to_sql('review', db, index=True, if_exists='replace', index_label=reviews.index.names)

In [ ]:
reviews.head()

In [ ]:
reviews.loc['Wilco'].sort_index()

In [ ]:
reviews.loc['Wilco'].album_score.mean().round(2)

Note how the new `review` table has fewer columns -- e.g. we no longer have genre. 

# Results

So, we have create four primary tables from our raw data.

In [ ]:
artists.head()

In [ ]:
albums.head()

In [ ]:
genres.head()

In [ ]:
reviews.head()

# Use `.join()` 

If we want to see genre in the review, we can create a join.

In [ ]:
reviews.join(albums)

In [ ]:
albums.join(artists, rsuffix='_artist')